add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
import asyncio

# --- Parameters ---
symbol = 'GOOGL'
expiry = '20250725'
strike_long = 175.0
strike_short = 185.0
right = 'P'
barSize = '5 mins'
duration = '7 D'
window = 20
num_std = 2
quantity = 1

# --- Connect to IBKR ---
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

# --- Define and qualify option contracts ---
option_buy = Option(symbol, expiry, strike_short, right, 'SMART')
option_sell = Option(symbol, expiry, strike_long, right, 'SMART')
await ib.qualifyContractsAsync(option_buy, option_sell)

# --- Request live historical bars ---
bars_buy = ib.reqHistoricalData(
    option_buy, '', duration, barSize, 'TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)
bars_sell = ib.reqHistoricalData(
    option_sell, '', duration, barSize, 'TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)

# --- Interactive Plotly FigureWidget for Jupyter ---
fig = go.FigureWidget()
fig.add_scatter(name='Spread', line=dict(color='black'))
fig.add_scatter(name='BB Upper', line=dict(color='green', dash='dash'))
fig.add_scatter(name='BB Lower', line=dict(color='red', dash='dash'))
fig.add_scatter(name='BB Middle', line=dict(color='orange', dash='dot'))
fig.add_scatter(name='Buy Signal', mode='markers', marker=dict(symbol='triangle-up', color='green', size=12))
fig.add_scatter(name='Sell Signal', mode='markers', marker=dict(symbol='triangle-down', color='red', size=12))
fig.update_layout(
    title='Live Combo Spread with Bollinger Bands and Signals',
    xaxis_title='Time',
    yaxis_title='Spread',
    template='plotly_white',
    width=1000,
    height=500
)
display(fig)  # <--- This line makes the interactive plot appear in Jupyter

# --- Combo and order utility functions ---
def make_combo_contract(buy_leg, sell_leg, is_long=True):
    combo = Contract()
    combo.symbol = buy_leg.symbol
    combo.secType = 'BAG'
    combo.currency = buy_leg.currency
    combo.exchange = 'SMART'
    combo.comboLegs = [
        ComboLeg(conId=buy_leg.conId, ratio=1, action='BUY' if is_long else 'SELL', exchange='SMART'),
        ComboLeg(conId=sell_leg.conId, ratio=1, action='SELL' if is_long else 'BUY', exchange='SMART')
    ]
    return combo

def calc_midprice():
    price1 = bars_buy[-1].close
    price2 = bars_sell[-1].close
    return round(price1 - price2, 2)

def make_combo_order(action, qty, limit_price):
    order = Order(
        action=action, orderType='LMT',
        totalQuantity=qty, lmtPrice=limit_price,
        tif='FOK',
        smartComboRoutingParams=[TagValue('NonGuaranteed', '1')]
    )
    return order

# --- Trade logic and live updating ---
pending_entry = None

def update_chart_and_logic():
    global pending_entry

    # --- Prepare Data ---
    df1 = util.df(bars_buy).set_index('date').rename(columns={'close': 'Close1'})
    df2 = util.df(bars_sell).set_index('date').rename(columns={'close': 'Close2'})
    data = df1.join(df2, how='inner', lsuffix='_1', rsuffix='_2')
    data['spread'] = data['Close1'] - data['Close2']
    data['bb_middle'] = data['spread'].rolling(window).mean()
    data['bb_std'] = data['spread'].rolling(window).std()
    data['bb_upper'] = data['bb_middle'] + num_std * data['bb_std']
    data['bb_lower'] = data['bb_middle'] - num_std * data['bb_std']
    data['prev_spread'] = data['spread'].shift(1)
    data['prev_bb_lower'] = data['bb_lower'].shift(1)
    data['prev_bb_upper'] = data['bb_upper'].shift(1)
    data['bb_signal'] = 'hold'
    buy_mask = (data['prev_spread'] > data['prev_bb_lower']) & (data['spread'] < data['bb_lower'])
    sell_mask = (data['prev_spread'] < data['prev_bb_upper']) & (data['spread'] > data['bb_upper'])
    data.loc[buy_mask, 'bb_signal'] = 'buy'
    data.loc[sell_mask, 'bb_signal'] = 'sell'

    # --- Trade Logic ---
    if not data.empty:
        last = data.iloc[-1]
        signal = last['bb_signal']
        print(f"[{last.name}] Spread = {last['spread']:.2f} | Signal = {signal}")

        # Position detection by contract ID
        positions = {p.contract.conId: p.position for p in ib.positions()}
        pos_buy = positions.get(option_buy.conId, 0)
        pos_sell = positions.get(option_sell.conId, 0)
        if pos_buy == 1 and pos_sell == -1:
            current_pos = 'long'
        elif pos_buy == -1 and pos_sell == 1:
            current_pos = 'short'
        else:
            current_pos = 'flat'

        mid = calc_midprice()

        # Flip/close or open positions with combo order
        if current_pos == 'long' and signal == 'sell':
            combo = make_combo_contract(option_buy, option_sell, is_long=False)
            order = make_combo_order('SELL', quantity, mid)
            ib.placeOrder(combo, order)
            pending_entry = 'short'
        elif current_pos == 'short' and signal == 'buy':
            combo = make_combo_contract(option_buy, option_sell, is_long=True)
            order = make_combo_order('BUY', quantity, mid)
            ib.placeOrder(combo, order)
            pending_entry = 'long'
        elif current_pos == 'flat':
            if pending_entry is not None:
                is_long = True if pending_entry == 'long' else False
                combo = make_combo_contract(option_buy, option_sell, is_long)
                order = make_combo_order('BUY' if is_long else 'SELL', quantity, mid)
                ib.placeOrder(combo, order)
                pending_entry = None
            else:
                if signal == 'buy':
                    combo = make_combo_contract(option_buy, option_sell, is_long=True)
                    order = make_combo_order('BUY', quantity, mid)
                    ib.placeOrder(combo, order)
                elif signal == 'sell':
                    combo = make_combo_contract(option_buy, option_sell, is_long=False)
                    order = make_combo_order('SELL', quantity, mid)
                    ib.placeOrder(combo, order)

    # --- Live chart update ---
    plot_df = data.tail(50)
    buy_signals = plot_df[plot_df['bb_signal'] == 'buy']
    sell_signals = plot_df[plot_df['bb_signal'] == 'sell']
    with fig.batch_update():
        fig.data[0].x = plot_df.index
        fig.data[0].y = plot_df['spread']
        fig.data[1].x = plot_df.index
        fig.data[1].y = plot_df['bb_upper']
        fig.data[2].x = plot_df.index
        fig.data[2].y = plot_df['bb_lower']
        fig.data[3].x = plot_df.index
        fig.data[3].y = plot_df['bb_middle']
        fig.data[4].x = buy_signals.index
        fig.data[4].y = buy_signals['spread']
        fig.data[5].x = sell_signals.index
        fig.data[5].y = sell_signals['spread']

# --- Live update event listener ---
def on_new_bar(bars, has_new_bar):
    if has_new_bar:
        update_chart_and_logic()

bars_buy.updateEvent += on_new_bar
bars_sell.updateEvent += on_new_bar

print("Running live combo strategy with FOK orders and interactive plotly widget...")
await asyncio.sleep(1e6)


FigureWidget({
    'data': [{'line': {'color': 'black'},
              'name': 'Spread',
              'type': 'scatter',
              'uid': '9620c862-bf9d-47a7-8141-88c275df3f0b'},
             {'line': {'color': 'green', 'dash': 'dash'},
              'name': 'BB Upper',
              'type': 'scatter',
              'uid': '40265652-15c5-4c13-adc7-3f24a6be887a'},
             {'line': {'color': 'red', 'dash': 'dash'},
              'name': 'BB Lower',
              'type': 'scatter',
              'uid': '8fd7831a-5e0b-4852-aa3b-d705865a3abf'},
             {'line': {'color': 'orange', 'dash': 'dot'},
              'name': 'BB Middle',
              'type': 'scatter',
              'uid': 'ed08d88c-2887-4d7b-a5bc-5f190b0ffea3'}],
    'layout': {'height': 500,
               'template': '...',
               'title': {'text': 'Live Spread with Bollinger Bands'},
               'width': 1000,
               'xaxis': {'title': {'text': 'Time'}},
               'yaxis': {'title': {'te

ValueError: columns overlap but no suffix specified: Index(['open', 'high', 'low', 'volume', 'average', 'barCount'], dtype='object')